<a href="https://colab.research.google.com/github/ghisford/Machine_learning/blob/main/MLOps_zenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install "zenml[server]"
!zenml integration install sklearn -y
!pip install pyparsing==2.4.7 # required for colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart= True)

NumExpr defaulting to 2 threads.
⠇ Installing integrations...


{'status': 'ok', 'restart': True}

In [27]:
!pip install kaleido


In [1]:
!pip install sqlalchemy>=2.0

In [ ]:
!pip install tensorflow-probability --upgrade

In [ ]:

!pip install typing-extensions==4.7.0

On colab, you need an ngrok account to view some of the visualizations later.
Create an account and copy your authtoken

# Example Experimentation ML Code
This is how you would normally write your ML code and feel like a superstar

In [2]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None:
  """ Train and test a Scikit-learn SVC classifier on digits"""
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size= 0.2,
                                                      shuffle = False)
  model = SVC(gamma = 0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9583333333333334


# Turning experiments into ML pipelines with ZenML
In practice, ML workflows are much more complicated than that. you need to write code that you will use multiple times for this like preprocessing, and evaluation using different datasets and models.

In [3]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.50.0
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Ini

In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"]
]:

  """ Load the digits dataset as numpy arrays """
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target,
                                                      test_size= 0.2, shuffle= False)

  return X_train, X_test, y_train, y_test




@step

def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
  """Train an sklearn SVC classifier"""
  model = SVC(gamma= 0.001)
  model.fit(X_train, y_train)
  return model


@step

def evaluator(
 X_test: np.ndarray,
 y_test: np.ndarray,
 model: ClassifierMixin,

) -> float:
  """Calculate the test set accuracy of an sklearn model"""
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")
  return test_acc


1. Next we use ZenML's pipeline decorator to connect all of our steps into an ML pipeline.
2. The pipeline definition does not depend on the concrete step functions we defined above. It merely establishes a recipe for how data moves through the steps.
3. This means we can replace steps as we wish e.g. run the pipeline with different models to compare their performance

In [5]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Links all the steps together in a pipeline"""
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train= X_train, y_train= y_train)
  evaluator(X_test= X_test, y_test= y_test, model= model)



# Running ZenML Pipelines
Finally, we initialize our pipeline with concrete step functions and call the run() method to run it

In [23]:
digits_svc_pipeline= digits_pipeline()
digits_svc_pipeline.run(unlisted= True)

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered version: (version: 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Linking artifact output to model None version None implicitly.
Step svc_trainer has started.
Using cached version of evaluator.
Linking artifact output to model None version None implicitly.
Step evaluator has started.
Run digits_pipeline-2023_12_07-18_19_04_919840 has finished in 0.404s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


AttributeError: ignored

## CONGRATULATIONS! YOU ARE THE INSPIRATION OF THE YOUTH, VOICE OF THE MONEY, THE KINGKONG.
That, my friend, was your first ever pipeline

Let us visualize the pipeline we have just run in zenml's dashboard. To do so, run
##zenml up
to spin up a zenml dashboard locally.
log in with username "default" and empty password and navigate to the Runs' tab in the pipelines section

In [19]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {2ZDxA4sbG2R6D4tOBT5kcwOw27g_5mzhhxuRUD1hevmY8F8XW}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
!ngrok config add-authtoken 2ZDxA4sbG2R6D4tOBT5kcwOw27g_5mzhhxuRUD1hevmY8F8XW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [24]:
from zenml.environment import Environment


def start_zenml_dashboard(port= 8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok
    public_url = ngrok.connect(port)
    print(r"\xlb[3lIn Colab, use this URL instead: {public_url}:\xrb[On")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-ca09df94-76e8-4199-995b-539b168cefc1


Opening tunnel named: http-8237-ca09df94-76e8-4199-995b-539b168cefc1


INFO:pyngrok.process.ngrok:t=2023-12-07T18:19:54+0000 lvl=info msg="no configuration paths supplied"


t=2023-12-07T18:19:54+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2023-12-07T18:19:54+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2023-12-07T18:20:37+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2023-12-07T18:20:37+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2023-12-07T18:20:37+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


INFO:pyngrok.process.ngrok:t=2023-12-07T18:20:37+0000 lvl=info msg="accept failed" obj=tunnels.session obj=csess id=17904b327b89 err="reconnecting session closed"


t=2023-12-07T18:20:37+0000 lvl=info msg="accept failed" obj=tunnels.session obj=csess id=17904b327b89 err="reconnecting session closed"
INFO:     Shutting down
INFO:     Finished server process [48421]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError

